## Initial Satelitte Data retrieval.

The following dataset was gathered from NASA FIRMs website and encases all fire anomalies throughout the year of 2019. The initial EDA that follows will narrow down the scope of our search to Northern California using the proper longitute and latitude ranges comprising a square area of approximately 70,000 km^2. All anomalies contained in the final dataframe should be over land, and also with a confidence rating of over 75%. Let's razzle dazzle.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import seaborn as sns
#import geopandas as pd
#import folium

import requests
from IPython.display import Image, display

import random

import urllib.request

import warnings
warnings.filterwarnings('ignore')

#show all columns

pd.set_option('display.max_columns', None)

In [2]:
modis_df= pd.read_csv('../data/modis_2019_United_States.csv')

In [3]:
modis_df.shape

(107213, 15)

In [4]:
modis_df.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,36.1292,-114.9293,304.2,1.2,1.1,2019-01-01,950,Aqua,MODIS,59,6.03,271.1,14.1,N,0
1,30.8237,-83.6847,318.0,1.8,1.3,2019-01-01,1600,Terra,MODIS,76,6.03,293.9,30.9,D,0
2,30.8298,-83.6779,310.6,1.8,1.3,2019-01-01,1600,Terra,MODIS,54,6.03,293.9,15.7,D,0
3,26.7338,-81.0102,327.6,1.1,1.1,2019-01-01,1601,Terra,MODIS,83,6.03,297.4,26.2,D,0
4,26.7258,-81.0230,313.3,1.1,1.1,2019-01-01,1601,Terra,MODIS,67,6.03,294.4,9.2,D,0


In [5]:
modis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107213 entries, 0 to 107212
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   latitude    107213 non-null  float64
 1   longitude   107213 non-null  float64
 2   brightness  107213 non-null  float64
 3   scan        107213 non-null  float64
 4   track       107213 non-null  float64
 5   acq_date    107213 non-null  object 
 6   acq_time    107213 non-null  int64  
 7   satellite   107213 non-null  object 
 8   instrument  107213 non-null  object 
 9   confidence  107213 non-null  int64  
 10  version     107213 non-null  float64
 11  bright_t31  107213 non-null  float64
 12  frp         107213 non-null  float64
 13  daynight    107213 non-null  object 
 14  type        107213 non-null  int64  
dtypes: float64(8), int64(3), object(4)
memory usage: 12.3+ MB


In [6]:
modis_df.isna().sum().sum()

0

In [7]:
new_df= modis_df[(modis_df['latitude'] >= 38.0881) & (modis_df['latitude'] <= 40.8336)]

In [8]:
new_df.shape

(8541, 15)

In [9]:
df_2= new_df[(new_df['longitude'] >= -123.1208) & (new_df['longitude'] <= -120.2933)]

In [10]:
df_2.shape

(2730, 15)

In [11]:
df_3 = df_2[(df_2.confidence >= 70)]

In [12]:
df_3.shape

(1537, 15)

### 

In [13]:
mod_2019_final = df_3

In [14]:
mod_2019_final.shape

(1537, 15)

In [15]:
 ### after brief research this last step is not necessary. Terra and Aqua satellite do not measure between fires on
#land or on water, but rather refer two different satellites that occupy two different orbitals around the earth.

At this point after doing my transformations i feel like i may need more data so lets import from 2015,2016,2017 and 2018 as well. considering we have 1537 isntances, this will not be enough images for our eventual CNN. But lets work our way back, from 2018

In [16]:
mod_2018 = pd.read_csv('../data/modis_2018_United_States.csv')

In [17]:
mod_2018_2= mod_2018[(mod_2018['latitude'] >= 38.0881) & (mod_2018['latitude'] <= 40.8336)]

In [18]:
mod_2018_3= mod_2018_2[(mod_2018_2['longitude'] >= -123.1208) & (mod_2018_2['longitude'] <= -120.2933)]

In [19]:
mod_2018_final = mod_2018_3[(mod_2018_3.confidence >= 70)]

In [20]:
mod_2018_final.shape

(6492, 15)

In [21]:
mod_2017 = pd.read_csv('../data/modis_2017_United_States.csv')

In [22]:
mod_2017_2= mod_2017[(mod_2017['latitude'] >= 38.0881) & (mod_2017['latitude'] <= 40.8336)]

In [23]:
mod_2017_3= mod_2017_2[(mod_2017_2['longitude'] >= -123.1208) & (mod_2017_2['longitude'] <= -120.2933)]

In [24]:
mod_2017_final = mod_2017_3[(mod_2017_3.confidence >= 70)]

In [25]:
mod_2017_final.shape

(1807, 15)

In [26]:
1537 + 6492 + 1807 + 576

10412

In [27]:
mod_2016 = pd.read_csv('../data/modis_2016_United_States.csv')

In [28]:
mod_2016_2 =mod_2016[(mod_2016['latitude'] >= 38.0881) & (mod_2016['latitude'] <= 40.8336)]

In [29]:
mod_2016_3= mod_2016_2[(mod_2016_2['longitude'] >= -123.1208) & (mod_2016_2['longitude'] <= -120.2933)]

In [30]:
mod_2016_final = mod_2016_3[(mod_2016_3.confidence >= 70)] 

In [31]:
mod_2016_final.shape

(576, 15)

In [32]:
frames= [mod_2016_final, mod_2017_final, mod_2018_final, mod_2019_final]
final_df= pd.concat(frames)

In [33]:
final_df.shape

(10412, 15)

In [34]:
final_df.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
6719,40.5313,-122.9687,313.1,1.4,1.2,2016-02-11,2138,Aqua,MODIS,73,6.2,286.5,19.4,D,0
6720,40.5297,-122.9847,325.9,1.4,1.2,2016-02-11,2138,Aqua,MODIS,84,6.2,285.4,38.8,D,0
7550,38.9585,-122.1252,311.5,1.0,1.0,2016-02-12,1903,Terra,MODIS,71,6.2,285.1,10.8,D,0
7570,40.5257,-123.0070,323.3,2.4,1.5,2016-02-12,2043,Aqua,MODIS,76,6.2,286.0,81.5,D,0
7571,40.5320,-122.9801,327.5,2.4,1.5,2016-02-12,2043,Aqua,MODIS,81,6.2,284.6,99.2,D,0


In [35]:
final_df.rename(columns={'latitude':'lat', 
                         'longitude':'lon',
                         'acq_date':'date'}, inplace = True)

In [36]:
final_df.shape

(10412, 15)

In [37]:
df_fire_initial= final_df[['date','lat','lon']]

In [38]:
df_fire_initial['date'] = df_fire_initial['date'].astype(str)

In [39]:
df_fire_initial['lon'] = df_fire_initial['lon'].astype(str)
df_fire_initial['lat'] = df_fire_initial['lat'].astype(str)

In [40]:
df_fire_initial.dtypes


date    object
lat     object
lon     object
dtype: object

In [41]:
df_fire_initial.shape
df_fire_initial.head()

,date,lat,lon
6719,2016-02-11,40.5313,-122.9687
6720,2016-02-11,40.5297,-122.9847
7550,2016-02-12,38.9585,-122.1252
7570,2016-02-12,40.5257,-123.007
7571,2016-02-12,40.532,-122.9801


## Getting the satelitte imagery
We now have all the fire instances we need to get all the images we need when we query nasa's LandSAT api.

## URL for the API:
example api 
https://api.nasa.gov/planetary/earth/imagery?lon=100.75&lat=1.5&date=2014-02-01&api_key=DEMO_KEY

In [42]:
key=open('../nasa_api/nasa_api_key.txt','r').read()

In [47]:
import time

In [48]:
from tqdm import tqdm

In [49]:
import os
os.path.join(os.path.pardir,'images','fire_images',)

'..\\images\\fire_images'

In [50]:
with tqdm(total=100) as pbar:
    for i in range(10):
        time.sleep(0.1)
        pbar.update(10)

100%|██████████| 100/100 [00:01<00:00, 98.99it/s]


## Setting up download request

In [51]:
#a = 'https://api.nasa.gov/planetary/earth/imagery?'
#b='lon='
#c='&lat='
#l= '&dim=0.025'
#d= '&date='
#e= '&api_key='


#i = 0
#with tqdm(total=df_fire_initial.shape[0]) as pbar:
    
    
#    for index, row in df_fire_initial.iterrows():
#        url= a + b + row['lon'] + c + row['lat'] + l + d + row['date'] + e + key
#        urllib.request.urlretrieve(url, os.path.join(os.path.pardir,'images','fire_images',)\
#                                    +row['lon'] + row['lat']+ row['date']
#                                    +'.jpg')
#        i += 1
#        pbar.update(1)
#        if i == 999:
#            time.sleep(3600)
#            i = 0
    

## Images downloaded are terrible, lets try google maps api


In [52]:
df_fire_initial['center']= df_fire_initial[['lat','lon']].agg(','.join, axis = 1)

In [53]:
df_fire_initial

,date,lat,lon,center
6719,2016-02-11,40.5313,-122.9687,"40.5313,-122.9687"
6720,2016-02-11,40.5297,-122.9847,"40.5297,-122.9847"
7550,2016-02-12,38.9585,-122.1252,"38.9585,-122.1252"
7570,2016-02-12,40.5257,-123.007,"40.5257,-123.007"
7571,2016-02-12,40.532,-122.9801,"40.532,-122.9801"
...,...,...,...,...
104060,2019-12-04,38.5259,-120.4962,"38.5259,-120.4962"
105137,2019-12-05,38.4384,-120.4773,"38.4384,-120.4773"
105208,2019-12-06,38.872,-120.6605,"38.872,-120.6605"
105209,2019-12-06,38.5313,-120.5913,"38.5313,-120.5913"


In [54]:
img_size = '350x350' # Let's try this for now.  Gets multiplied by scale for resolution
# 350x350 seems to be about the minimum for image resolution
# ~100kb each, 20k images, 2 gigs for the batch, wf and nwf

img_format = 'jpg' # For compressability (is that a word?) png is 'png8' for 8 bit and 'png32' for 32 bit

map_scale = '1' # For scale parameter. 

maptype = 'satellite' # Obvious reason

zoom = '15' # try that to start

In [55]:
key_2 = open('../google_api/google_key.txt', 'r').read()

In [56]:
a = 'https://maps.googleapis.com/maps/api/staticmap?' # Base
b = 'center=' # Center 
# Enter Center
c = '&zoom=' # Zoom
# Enter Zoom
d = '&maptype=satellite' # Map type 
# No need to enter maptype - just keep satellite default
e = '&size=' # Image Size
# Enter image size
f = '&key='
# Enter key

# Creating the URL:
url1 = a + b
url2 = c + zoom + d + e + img_size + f + key_2
# URL = url1 + row['center'] + url2

In [57]:
df_fire_initial.index[6370:].shape[0] + 6370

10412


for index, row in df_fire_initial.iterrows():
    url = url1 + row['center'] + url2
    r = requests.get(url)
    display(Image(r.content))

In [57]:
#i = 0 
#with tqdm(total=df_fire_initial.shape[0]) as pbar:

 #   for index[:6369], row in df_fire_initial.iterrows():
  #      url= url1 + row['center'] + url2
   #     urllib.request.urlretrieve(url, os.path.join(os.path.pardir,'images','fire_images',)\
    #                                + row['center']
     #                               +'.jpg')
      #      #i += 1
       # pbar.update(1)
            #if i == 999:
                #time.sleep(3600)
                #i = 0

 61%|██████    | 6370/10412 [1:02:19<39:32,  1.70it/s]


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

The loop stopped when it was 61% through. I am going to max a new dataframe based off of where the progress bar indicates that it stopped.

In [58]:
#df_fire_2 = df_fire_initial.index[6369]
#df_fire_2

81163

In [64]:
#with tqdm(total=df_fire_initial.shape[0]) as pbar:

 #   for index, row in df_fire_initial.iterrows():
  #      url= url1 + row['center'] + url2
   #     urllib.request.urlretrieve(url, os.path.join(os.path.pardir,'images','fire_images',)\
                                    + row['center']
                                    +'.jpg')
            #i += 1
    #    pbar.update(1)
            #if i == 999:
                #time.sleep(3600)
                #i = 0

  0%|          | 0/10412 [00:00<?, ?it/s]


TypeError: 'generator' object is not subscriptable

## Retrieving Non-fire images

this is going to be a function that iterates over the same area, and pulls out non fire classes.

In [64]:
non_fire_size= 10000

df_fire_initial['lat'] = df_fire_initial.lat.astype(float) #to get randomized lats and lons we need to convert back to floats
df_fire_initial['lon'] = df_fire_initial.lon.astype(float)

new_lat = np.random.uniform(low= min(df_fire_initial.lat),
                            high = max(df_fire_initial.lat),  #randomizing our new coordinated between the square area we
                            size= (non_fire_size,))         # retrieved the wildfire images from

new_lon = np.random.uniform(low = min(df_fire_initial.lon),
                            high = max(df_fire_initial.lon),
                            size=(non_fire_size,))

new_coordinates= {'lat':new_lat,'lon':new_lon}

df_non_fire = pd.DataFrame(data = new_coordinates)


df_non_fire['lat'] = df_non_fire['lat'].astype(str)  #converting our new coordinates to string so we can use them when we call
df_non_fire['lon'] = df_non_fire['lon'].astype(str)  # on our API
df_non_fire['center'] = df_non_fire[['lat', 'lon']].agg(','.join, axis = 1) 



df_fire_initial['lat'] = df_fire_initial.lat.astype(str)
df_fire_initial['lon'] = df_fire_initial.lon.astype(str)

df_non_fire.head()



,lat,lon,center
0,38.704497550672336,-122.44739100683832,"38.704497550672336,-122.44739100683832"
1,40.822938312887686,-121.67970310729866,"40.822938312887686,-121.67970310729866"
2,38.39717636294755,-122.99953069633972,"38.39717636294755,-122.99953069633972"
3,38.94127182813535,-122.03948981727733,"38.94127182813535,-122.03948981727733"
4,39.4018715439874,-121.40084057280517,"39.4018715439874,-121.40084057280517"


In [60]:
os.path.join(os.path.pardir,'images','non_fire',)

'..\\images\\non_fire'

In [65]:
#i = 0 
with tqdm(total=df_non_fire.shape[0]) as pbar:

    for index, row in df_non_fire.iterrows():
        url= url1 + row['center'] + url2
        urllib.request.urlretrieve(url, os.path.join(os.path.pardir,'images','non_fire',)\
                                    + row['center']
                                    +'.jpg')
            
        pbar.update(1)
            

 11%|█▏        | 1136/10000 [04:12<32:47,  4.50it/s]


KeyboardInterrupt: 